In [71]:
import numpy as np
%%capture
!pip install pandas nltk scikit-learn

UsageError: Line magic function `%%capture` not found.


In [72]:
import pandas as pd
import nltk
from preprocessing import lowercase_data, remove_punctuation, remove_stopwords, LemmatizerOrStemmer, WordSimplification
from typing import Tuple, Dict, List, Union, Set

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/serms/.config/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/serms/.config/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/serms/.config/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [73]:
dtypes = {
    'ArticleId': 'int32',
    'Text': 'str',
    'Category': 'category'
}
data_train = pd.read_csv('data/BBC News Train.csv', dtype=dtypes, encoding='utf-8', index_col='ArticleId')
data_test = pd.read_csv('data/BBC News Test.csv', dtype={'ArticleId': 'int32', 'Text': 'str'}, encoding='utf-8', index_col='ArticleId')
data_test_solution = pd.read_csv('data/BBC News Sample Solution.csv', dtype={'ArticleId': 'int32', 'Category': 'category'}, encoding='utf-8', index_col='ArticleId')
data_test['Category'] = data_test_solution['Category']
data = pd.concat([data_train, data_test])
data.head(10)

,Text,Category
ArticleId,,
1833,worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness. cynthia cooper worldcom s ex-head of internal accounting alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy. prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper who now runs her own consulting business told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a green light to the procedures and practices used by worldcom. mr ebber s lawyers have said he was unaware of the fraud arguing that auditors did not alert him to any problems. ms cooper also said that during shareholder meetings mr ebbers often passed over technical questions to the company s finance chief giving only brief answers himself. the prosecution s star witness former worldcom financial chief scott sullivan has said that mr ebbers ordered accounting adjustments at the firm telling him to hit our books . however ms cooper said mr sullivan had not mentioned anything uncomfortable about worldcom s accounting during a 2001 audit committee meeting. mr ebbers could face a jail sentence of 85 years if convicted of all the charges he is facing. worldcom emerged from bankruptcy protection in 2004 and is now known as mci. last week mci agreed to a buyout by verizon communications in a deal valued at $6.75bn.,business
154,german business confidence slides german business confidence fell in february knocking hopes of a speedy recovery in europe s largest economy. munich-based research institute ifo said that its confidence index fell to 95.5 in february from 97.5 in january its first decline in three months. the study found that the outlook in both the manufacturing and retail sectors had worsened. observers had been hoping that a more confident business sector would signal that economic activity was picking up. we re surprised that the ifo index has taken such a knock said dz bank economist bernd weidensteiner. the main reason is probably that the domestic economy is still weak particularly in the retail trade. economy and labour minister wolfgang clement called the dip in february s ifo confidence figure a very mild decline . he said that despite the retreat the index remained at a relatively high level and that he expected a modest economic upswing to continue. germany s economy grew 1.6% last year after shrinking in 2003. however the economy contracted by 0.2% during the last three months of 2004 mainly due to the reluctance of consumers to spend. latest indications are that growth is still proving elusive and ifo president hans-werner sinn said any improvement in german domestic demand was sluggish. exports had kept things going during the first half of 2004 but demand for exports was then hit as the value of the euro hit record levels making german products less competitive overseas. on top of that the unemployment rate has been stuck at close to 10% and manufacturing firms including daimlerchrysler siemens and volkswagen have been negotiating with unions over cost cutting measures. analysts said that the ifo figures and germany s continuing problems may delay an interest rate rise by the european central bank. eurozone interest rates are at 2% but comments from senior officials have recently focused on the threat of inflation prompting fears that interest rates may rise.,business
1101,bbc poll indicates economic gloom citizens in a majority of nations surveyed in a bbc world service pol

In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2225 entries, 1833 to 471
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Text      2225 non-null   object  
 1   Category  2225 non-null   category
dtypes: category(1), object(1)
memory usage: 28.5+ KB


In [75]:
data.Category.value_counts()

Category
sport            493
business         483
politics         421
entertainment    420
tech             408
Name: count, dtype: int64

## Preprocessing

In [76]:
def data_preprocessing(data: Union[pd.Series, List[str], str], stopwords: Set[str]) -> Union[pd.Series, List[str], str]:
    data = lowercase_data(data)
    data = remove_punctuation(data)
    data = remove_stopwords(data, stopwords)
    return data

In [77]:
from nltk.corpus import stopwords

english_stopwords = set(stopwords.words('english'))

data['Text'] = data_preprocessing(data['Text'], english_stopwords)
data.head(10)

,Text,Category
ArticleId,,
1833,worldcom ex boss launches defence lawyers defending former worldcom chief bernie ebbers battery fraud charges called company whistleblower first witness cynthia cooper worldcom ex head internal accounting alerted directors irregular accounting practices us telecoms giant warnings led collapse firm following discovery bn bn accounting fraud mr ebbers pleaded guilty charges fraud conspiracy prosecution lawyers argued mr ebbers orchestrated series accounting tricks worldcom ordering employees hide expenses inflate revenues meet wall street earnings estimates ms cooper runs consulting business told jury new york wednesday external auditors arthur andersen approved worldcom accounting early said andersen given green light procedures practices used worldcom mr ebber lawyers said unaware fraud arguing auditors alert problems ms cooper also said shareholder meetings mr ebbers often passed technical questions company finance chief giving brief answers prosecution star witness former worldcom financial chief scott sullivan said mr ebbers ordered accounting adjustments firm telling hit books however ms cooper said mr sullivan mentioned anything uncomfortable worldcom accounting audit committee meeting mr ebbers could face jail sentence years convicted charges facing worldcom emerged bankruptcy protection known mci last week mci agreed buyout verizon communications deal valued bn,business
154,german business confidence slides german business confidence fell february knocking hopes speedy recovery europe largest economy munich based research institute ifo said confidence index fell february january first decline three months study found outlook manufacturing retail sectors worsened observers hoping confident business sector would signal economic activity picking surprised ifo index taken knock said dz bank economist bernd weidensteiner main reason probably domestic economy still weak particularly retail trade economy labour minister wolfgang clement called dip february ifo confidence figure mild decline said despite retreat index remained relatively high level expected modest economic upswing continue germany economy grew last year shrinking however economy contracted last three months mainly due reluctance consumers spend latest indications growth still proving elusive ifo president hans werner sinn said improvement german domestic demand sluggish exports kept things going first half demand exports hit value euro hit record levels making german products less competitive overseas top unemployment rate stuck close manufacturing firms including daimlerchrysler siemens volkswagen negotiating unions cost cutting measures analysts said ifo figures germany continuing problems may delay interest rate rise european central bank eurozone interest rates comments senior officials recently focused threat inflation prompting fears interest rates may rise,business
1101,bbc poll indicates economic gloom citizens majority nations surveyed bbc world service poll believe world economy worsening respondents also said national economy getting worse asked family financial outlook majority countries said positive future almost people countries questioned poll mostly conducted asian tsunami disaster poll found majority plurality people countries believed economy going downhill compared respondents nine countries believed improving surveyed three countries split percentage terms average respondents country said world economy getting worse compared said improving similarly pessimistic national economy optimistic saw family economic conditions improving said getting worse poll people conducted international polling firm globescan together program international policy attitudes pipa university maryland world economy picked difficult times years ago people seem fully absorbed development though personally experiencing effects said pipa director steven kull people around world saying ok world may perception war terrorism religious polit

In [78]:
word_simplifier_type = WordSimplification.LEMMATIZER
word_simplifier = LemmatizerOrStemmer(WordSimplification.LEMMATIZER)

data['Text'] = word_simplifier.simplify(data['Text'])
data.head(10)

,Text,Category
ArticleId,,
1833,worldcom ex boss launches defence lawyers defending former worldcom chief bernie ebbers battery fraud charges called company whistleblower first witness cynthia cooper worldcom ex head internal accounting alerted directors irregular accounting practices us telecoms giant warnings led collapse firm following discovery bn bn accounting fraud mr ebbers pleaded guilty charges fraud conspiracy prosecution lawyers argued mr ebbers orchestrated series accounting tricks worldcom ordering employees hide expenses inflate revenues meet wall street earnings estimates ms cooper runs consulting business told jury new york wednesday external auditors arthur andersen approved worldcom accounting early said andersen given green light procedures practices used worldcom mr ebber lawyers said unaware fraud arguing auditors alert problems ms cooper also said shareholder meetings mr ebbers often passed technical questions company finance chief giving brief answers prosecution star witness former worldcom financial chief scott sullivan said mr ebbers ordered accounting adjustments firm telling hit books however ms cooper said mr sullivan mentioned anything uncomfortable worldcom accounting audit committee meeting mr ebbers could face jail sentence years convicted charges facing worldcom emerged bankruptcy protection known mci last week mci agreed buyout verizon communications deal valued bn,business
154,german business confidence slides german business confidence fell february knocking hopes speedy recovery europe largest economy munich based research institute ifo said confidence index fell february january first decline three months study found outlook manufacturing retail sectors worsened observers hoping confident business sector would signal economic activity picking surprised ifo index taken knock said dz bank economist bernd weidensteiner main reason probably domestic economy still weak particularly retail trade economy labour minister wolfgang clement called dip february ifo confidence figure mild decline said despite retreat index remained relatively high level expected modest economic upswing continue germany economy grew last year shrinking however economy contracted last three months mainly due reluctance consumers spend latest indications growth still proving elusive ifo president hans werner sinn said improvement german domestic demand sluggish exports kept things going first half demand exports hit value euro hit record levels making german products less competitive overseas top unemployment rate stuck close manufacturing firms including daimlerchrysler siemens volkswagen negotiating unions cost cutting measures analysts said ifo figures germany continuing problems may delay interest rate rise european central bank eurozone interest rates comments senior officials recently focused threat inflation prompting fears interest rates may rise,business
1101,bbc poll indicates economic gloom citizens majority nations surveyed bbc world service poll believe world economy worsening respondents also said national economy getting worse asked family financial outlook majority countries said positive future almost people countries questioned poll mostly conducted asian tsunami disaster poll found majority plurality people countries believed economy going downhill compared respondents nine countries believed improving surveyed three countries split percentage terms average respondents country said world economy getting worse compared said improving similarly pessimistic national economy optimistic saw family economic conditions improving said getting worse poll people conducted international polling firm globescan together program international policy attitudes pipa university maryland world economy picked difficult times years ago people seem fully absorbed development though personally experiencing effects said pipa director steven kull people around world saying ok world may perception war terrorism religious polit

In [99]:
user_interests_1: Dict[int, List[str]] = {
    1: ['politics'],
    2: ['entertainment'],
    3: ['sport'],
    4: ['tech'],
    5: ['business']
}

user_interests_2 = {
    1: ['politics', 'soccer'],
    2: ['music', 'films'],
    3: ['cars', 'politics'],
    4: ['soccer']
}

In [120]:
def flatten(xss):
    return [x for xs in xss for x in xs]

def preprocess_user_interests(user_interests: Dict[int, List[str]]) -> Dict[int, List[str]]:
    interests_processed = {}
    for user_id, interests in user_interests.items():
         interests_processed[user_id] = data_preprocessing(interests, english_stopwords)
    return interests_processed

num_documents_to_retrieve = 10  # Number of documents to retrieve for each user
user_interests = preprocess_user_interests(user_interests_1)
user_interests

{1: ['politics'],
 2: ['entertainment'],
 3: ['sport'],
 4: ['tech'],
 5: ['business']}

# TF-IDF

In [121]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

def get_tfidf(data: pd.Series, *, min_df: float = None, max_df: float = None, stop_words_language: str = 'english', max_features: int = 100) -> Tuple[TfidfVectorizer, csr_matrix]:
    data = data.fillna('')
    vectorizer = TfidfVectorizer(min_df=min_df or 0.0, max_df=max_df or 0.0,  stop_words=stop_words_language, max_features=max_features)
    vectors = vectorizer.fit_transform(data)
    return vectorizer, vectors

In [122]:
vectorizer, document_vectors = get_tfidf(data['Text'], max_df=0.05, stop_words_language='english', max_features=1000)

In [123]:
document_vectors.shape

(2225, 1000)

In [136]:
from sklearn.metrics.pairwise import cosine_similarity

user_retrieved_documents: Dict[int, pd.DataFrame] = {
    user_id: pd.DataFrame(
        index=data.index,
        columns=['SimilarityScore']
    )
    for user_id, _ in user_interests.items()
}

for user_id, interests in user_interests.items():
    print(f'User {user_id} have interests: [{', '.join(interests)}]')
    for interest in interests:
        query_vector = vectorizer.transform([interest])
        similarity_scores = cosine_similarity(query_vector, document_vectors)[0]
        print(f"Similarity scores: {similarity_scores}")
        top_documents_indices = similarity_scores.argsort()[-num_documents_to_retrieve:][::-1]
        for document_index in top_documents_indices:
            article_id = data.index[document_index]
            user_retrieved_documents[user_id].loc[article_id] = similarity_scores[document_index]
            print(f"Document {article_id} with similarity score {similarity_scores[document_index]}")
            
user_retrieved_documents[1]

User 1 have interests: [politics]
Similarity scores: [0. 0. 0. ... 0. 0. 0.]
Document 642 with similarity score 0.5950719428359025
Document 1792 with similarity score 0.578311405315986
Document 553 with similarity score 0.4665441969474489
Document 825 with similarity score 0.4449098397095696
Document 613 with similarity score 0.3428030118435868
Document 14 with similarity score 0.3279381543551124
Document 86 with similarity score 0.3279381543551124
Document 2211 with similarity score 0.3006464417554213
Document 1104 with similarity score 0.26068655104605937
Document 2204 with similarity score 0.2273985428836941
User 2 have interests: [entertainment]
Similarity scores: [0. 0. 0. ... 0. 0. 0.]
Document 985 with similarity score 0.5966060392716641
Document 1147 with similarity score 0.41303112274216996
Document 326 with similarity score 0.3283269672954306
Document 1493 with similarity score 0.26031826969412664
Document 992 with similarity score 0.23288658699423126
Document 332 with simila

,SimilarityScore
ArticleId,
1833,NaN
154,NaN
1101,NaN
1976,NaN
917,NaN
...,...
1923,NaN
373,NaN
1704,NaN
